In [2]:
import os
import torch
import torchvision
from tqdm.auto import tqdm

In [12]:
TOTAL_FRAME = 60
SOURCE_SCENES = ['00000', '00001', '00002', '00003', '00004']
TARGET_SCENES = [ '01237','00700','00321', '01425']
VERSION = range(4)
INPUT_DIR = "../../output/20240925/multi_mlp_fit/lightning_logs/version_{}/epoch_0049/"
OUTPUT_DIR = "../../output/20240925/multi_mlp_fit/lightning_logs/"


In [20]:

            
for version_id in tqdm(VERSION):
    images = []
    input_dir = INPUT_DIR.format(version_id)
    # first row is target_light 
    images.append(torch.zeros(3,256,256))
    images.append(torch.zeros(3,256,256))
    for scene in ['00000']:
        for target_scene in TARGET_SCENES:
            try:
                filename = f'{scene}_{target_scene}.jpg'
                image = torchvision.io.read_image(os.path.join(input_dir,'source_image', filename)) / 255.0
                # resize image to 256x256
                image = torchvision.transforms.functional.resize(image, (256,256))
                images.append(image)
            except:
                images.append(torch.zeros(3,256,256))
    for scene in SOURCE_SCENES:
        # first image is itself gt 
        try:
            image = torchvision.io.read_image(os.path.join(input_dir,'source_image', f'{scene}_{scene}.jpg')) / 255.0
            # resize image to 256x256
            image = torchvision.transforms.functional.resize(image, (256,256))
            images.append(image)
        except:
            images.append(torch.zeros(3,256,256))
        # second row is itself ddim
        try:
            image = torchvision.io.read_image(os.path.join(input_dir,'crop_image', f'{scene}_{scene}.jpg')) / 255.0
            # resize image to 256x256
            image = torchvision.transforms.functional.resize(image, (256,256))
            images.append(image)
        except:
            images.append(torch.zeros(3,256,256))

        # then for loop for target_scene
        for target_scene in TARGET_SCENES:
            try:
                filename = f'{scene}_{target_scene}.jpg'
                image = torchvision.io.read_image(os.path.join(input_dir,'crop_image', filename)) / 255.0
                # resize image to 256x256
                image = torchvision.transforms.functional.resize(image, (256,256))
                images.append(image)
            except:
                images.append(torch.zeros(3,256,256))
    # make grid
    grid = torchvision.utils.make_grid(images, nrow=2 + len(TARGET_SCENES))
    # save image
    output_path = os.path.join(OUTPUT_DIR, f'version_{version_id}.jpg')
    torchvision.utils.save_image(grid, output_path)

  0%|          | 0/4 [00:00<?, ?it/s]